This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers, callbacks
import numpy as np

c:\Users\heral\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.2 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\heral\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.2 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\heral\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.30.2 at tensorflow/core/framewo

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = 'C:/Users/heral/Downloads/Train'  # e.g. './muffin-vs-chihuahua/train'
test_dir = 'C:/Users/heral/Downloads/Test'    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# SIMPLE CNN MODEL ARCHITECTURE (IMPROVED)
import random
# reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Regularization and dropout settings
weight_decay = 1e-4

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\heral\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# optimizer with learning rate schedule + compile
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,177 (12.61 MB)

 Trainable params: 3,305,473 (12.61 MB)

 Non-trainable params: 704 (2.75 KB)

In [7]:
# TRAINING THE CNN (IMPROVED)
from tensorflow.keras import callbacks as cb

es = cb.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
rlp = cb.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[es, rlp]
)

# Evaluate on test set and print accuracy
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

# Save improved model with required filename
model.save('exercise_6_trained_model_improved.h5')
print('Saved model to exercise_6_trained_model_improved.h5')

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 325ms/step - accuracy: 0.7846 - loss: 0.5938 - val_accuracy: 0.5407 - val_loss: 3.5478 - learning_rate: 0.0010
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 325ms/step - accuracy: 0.7846 - loss: 0.5938 - val_accuracy: 0.5407 - val_loss: 3.5478 - learning_rate: 0.0010
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 319ms/step - accuracy: 0.8266 - loss: 0.4857 - val_accuracy: 0.5407 - val_loss: 2.5826 - learning_rate: 0.0010
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 319ms/step - accuracy: 0.8266 - loss: 0.4857 - val_accuracy: 0.5407 - val_loss: 2.5826 - learning_rate: 0.0010
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 315ms/step - accuracy: 0.8506 - loss: 0.4261 - val_accuracy: 0.5407 - val_loss: 1.8130 - learning_rate: 0.0010
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 315ms/step - accuracy: 0.8506 - loss: 0.4261 - val_accuracy: 0.5407 - val_loss: 1.8130 - learning_rate: 0.0010
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 321ms/step - accuracy: 0.8

Test Accuracy: 0.9113
Saved model to exercise_6_trained_model_improved.h5


In [8]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - accuracy: 0.9113 - loss: 0.3069
Test Accuracy: 0.9113175868988037
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - accuracy: 0.9113 - loss: 0.3069
Test Accuracy: 0.9113175868988037


In [9]:
model.save('exercise_6_trained_model_improved.h5')

In [10]:
# SIMPLE INFERENCE SCRIPT (returns confidence and can append to CSV)
from tensorflow.keras.preprocessing import image
import csv
import os

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5', write_csv=None):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = float(model.predict(img_array, verbose=0)[0,0])
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    print(f"{os.path.basename(img_path)} -> Prediction: {label} (confidence: {pred:.4f})")
    if write_csv:
        header_needed = not os.path.exists(write_csv) or os.path.getsize(write_csv) == 0
        with open(write_csv, 'a', newline='') as f:
            writer = csv.writer(f)
            if header_needed:
                writer.writerow(['image','label','confidence'])
            writer.writerow([os.path.basename(img_path), label, pred])
    return label, pred


In [11]:
# Example usage and capture Run predictions
image_paths = [
    'C:/Users/heral/Downloads/muff.jpg',
    'C:/Users/heral/Downloads/chi.jpg',
    'C:/Users/heral/Downloads/muff2.jpg',
    'C:/Users/heral/Downloads/chi2.jpg',
    'C:/Users/heral/Downloads/muff3.jpg'
]
run_csv = 'run_predictions.csv'
for p in image_paths:
    try:
        predict_image(p, model_path='exercise_6_trained_model_improved.h5', write_csv=run_csv)
    except Exception as e:
        print('Error predicting', p, e)

muff.jpg -> Prediction: Muffin (confidence: 0.8135)
chi.jpg -> Prediction: Chihuahua (confidence: 0.0004)
chi.jpg -> Prediction: Chihuahua (confidence: 0.0004)


muff2.jpg -> Prediction: Muffin (confidence: 0.9967)


chi2.jpg -> Prediction: Muffin (confidence: 0.5538)


muff3.jpg -> Prediction: Muffin (confidence: 0.9912)


In [ ]:
# 3) Train and evaluate on custom dataset: Normal vs Military Vehicle (self-contained)
import random
try:
    import tensorflow as tf
except Exception:
    raise
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers, callbacks
import os, csv

# Ensure IMG_SIZE and BATCH_SIZE exist; otherwise set defaults
try:
    IMG_SIZE
except NameError:
    IMG_SIZE = (128, 128)

try:
    BATCH_SIZE
except NameError:
    BATCH_SIZE = 32

tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

custom_train_dir = r"C:\Users\heral\Downloads\dataset\train"
custom_test_dir  = r"C:\Users\heral\Downloads\dataset\test"

train_datagen_c = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, validation_split=0.2)
test_datagen_c = ImageDataGenerator(rescale=1./255)

train_gen_c = train_datagen_c.flow_from_directory(custom_train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training')
val_gen_c   = train_datagen_c.flow_from_directory(custom_train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation')
test_gen_c  = test_datagen_c.flow_from_directory(custom_test_dir,  target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)

print('Train classes:', train_gen_c.class_indices)

# Build a simple student-friendly model for the custom dataset
wd = 1e-4
model_custom = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(wd), input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(wd)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(wd)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_custom.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model_custom.summary()

es = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rlp = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history_c = model_custom.fit(train_gen_c, epochs=15, validation_data=val_gen_c, callbacks=[es, rlp])

test_loss_c, test_acc_c = model_custom.evaluate(test_gen_c)
print(f"Custom dataset test accuracy: {test_acc_c:.4f}")

model_custom.save('exercise_6_custom_amolong.h5')
print('Saved exercise_6_custom_amolong.h5')

# Inference on the two provided images and save results
from tensorflow.keras.preprocessing import image
images_to_predict = [r"C:\Users\heral\Downloads\normal1.jpg", r"C:\Users\heral\Downloads\mili1.jpg"]
csv_path = 'run_custom_predictions.csv'
with open(csv_path, 'w', newline='') as f:
    csv.writer(f).writerow(['image','label','confidence'])

for p in images_to_predict:
    try:
        img = image.load_img(p, target_size=IMG_SIZE)
        arr = image.img_to_array(img) / 255.0
        arr = np.expand_dims(arr, axis=0)
        prob = float(model_custom.predict(arr, verbose=0)[0,0])
        label = 'Normal' if prob < 0.5 else 'Military'
        print(f"{os.path.basename(p)} -> {label} ({prob:.3f})")
        with open(csv_path, 'a', newline='') as f:
            csv.writer(f).writerow([os.path.basename(p), label, prob])
    except Exception as e:
        print('Error predicting', p, e)

Found 8936 images belonging to 2 classes.
Found 2234 images belonging to 2 classes.
Found 2234 images belonging to 2 classes.
Found 2792 images belonging to 2 classes.
Found 2792 images belonging to 2 classes.
Train classes: {'millitary': 0, 'other': 1}
Train classes: {'millitary': 0, 'other': 1}


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,706,305 (14.14 MB)

 Trainable params: 3,706,113 (14.14 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 62s 219ms/step - accuracy: 0.6758 - loss: 0.8276 - val_accuracy: 0.4893 - val_loss: 4.0264 - learning_rate: 0.0010
Epoch 2/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 62s 219ms/step - accuracy: 0.6758 - loss: 0.8276 - val_accuracy: 0.4893 - val_loss: 4.0264 - learning_rate: 0.0010
Epoch 2/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 63s 226ms/step - accuracy: 0.7003 - loss: 0.6139 - val_accuracy: 0.6643 - val_loss: 1.2073 - learning_rate: 0.0010
Epoch 3/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 63s 226ms/step - accuracy: 0.7003 - loss: 0.6139 - val_accuracy: 0.6643 - val_loss: 1.2073 - learning_rate: 0.0010
Epoch 3/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 64s 229ms/step - accuracy: 0.7292 - loss: 0.5505 - val_accuracy: 0.8057 - val_loss: 0.5172 - learning_rate: 0.0010
Epoch 4/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 64s 229ms/step - accuracy: 0.7292 - loss: 0.5505 - val_accuracy: 0.8057 - val_loss: 0.5172 - learning_rate: 0.0010
Epoch 4/15
280/280 ━━━━━━━━━━━━━━━━━━━━ 62s 222ms/step - accuracy: 0.7

Custom dataset test accuracy: 0.8499
Saved exercise_6_custom_amolong.h5
Saved exercise_6_custom_amolong.h5
normal1.jpg -> Military (0.923)
mili1.jpg -> Military (0.846)
normal1.jpg -> Military (0.923)
mili1.jpg -> Military (0.846)
